In [139]:
import pandas as pd
import requests
import os
import sys

#URLlib
import urllib
import urllib.request

#PDFminer
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import TextConverter
from io import StringIO
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBoxHorizontal

In [140]:
#Palabras a buscar 
# word_count = "Word Count"
# letter_count = "Letter Count"
word_1 = "Crecimiento"
word_2 = "Riesgo"
word_3 = "inflacion"
#Misma formato para mas palabras
words_to_seek = ["actividad", "económica", "mundial", "méxico"]

In [141]:
word_seek = pd.DataFrame( index = [word_count,
                                  letter_count,
                                  word_1,
                                  word_2,
                                  word_3])

In [142]:
path = "statement.pdf"

In [143]:
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,
                                  password=password,
                                  caching=caching,
                                  check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [144]:
pdf_text = convert_pdf_to_txt(path).replace('\n', '')

In [145]:
pdf_text = pdf_text.replace("\x0c", "")
pdf_text = pdf_text.lower()
pdf_text

' 14 de noviembre de 2019 anuncio de política monetaria  comunicado de prensa la junta de gobierno del banco de méxico ha decidido disminuir en 25 puntos base el objetivo para la tasa de interés interbancaria a un día a un nivel de 7.50%.  durante el tercer trimestre del año continuó la desaceleración de la economía mundial y sus perspectivas de crecimiento han seguido revisándose a la baja. ello, en un entorno en el que las inflaciones general y subyacente en las principales economías avanzadas permanecen en niveles inferiores a las metas de sus respectivos bancos centrales. en este contexto, algunas de estas instituciones han tomado acciones de relajamiento monetario en los últimos meses. en su reunión de octubre la reserva federal redujo en 25 puntos base el rango objetivo para la tasa de fondos federales y en su comunicación destacó que el rango  alcanzado  es  apropiado  considerando  los  riesgos  que  enfrenta.  por  su  parte,  los  mercados financieros  internacionales  presen

In [146]:
set_of_words_in_file = set(pdf_text.split(' '))
list_of_words_in_file = list(pdf_text.split(' '))

In [149]:
words_df = pd.DataFrame(list_of_words_in_file)
words_df.columns = ["words"]
words_df

,words
0,
1,14
2,de
3,noviembre
4,de
...,...
1594,dicha
1595,meta.
1596,2
1597,


In [148]:
count = 0
for word in words_to_seek:
    if word in list_of_words_in_file:
        print(f"{word} is in pdf 🎻")
    else:
        print(f"{word} isn't in pdf")

actividad is in pdf 🎻
económica is in pdf 🎻
mundial is in pdf 🎻
méxico is in pdf 🎻
